In [8]:
import torch
import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import copy
import h5py
import itertools
import os
import gc

from model.model_3D import *
from model.train_eval import *

from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

from postprocessing.save_results import *
from postprocessing.plot_results import *
from postprocessing.metrics import single_roc_curve

In [ ]:
### check if cuda is available

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using {device}.")

In [ ]:
### import NDVI data

def load_with_hdf5(name):
    """
    Load tensor data from HDF5 with gzip compression
    """
    with h5py.File(f"data/{name}.h5", "r") as f:
        loaded_features = torch.from_numpy(f["features"][:])
        loaded_labels = torch.from_numpy(f["labels"][:])
    return TensorDataset(loaded_features, loaded_labels)

train_set = load_with_hdf5("train_set")
val_set = load_with_hdf5("val_set")
test_set = load_with_hdf5("test_set")

In [ ]:
# check shape of input and target tensors

print(train_set.tensors[0].shape)
print(train_set.tensors[1].shape)


In [6]:
def train_and_validate(batch_size, learning_rate, init_hid_dim, num_epochs=100, device='cuda'):
    # Initialize model
    model = UNet3D(
        n_channels=train_set[0][0].shape[0],
        n_classes=1,
        init_hid_dim=init_hid_dim,
        kernel_size=3,
        pooling='max',
        bilinear=False,
        drop_channels=False
    )

    # Clear CUDA cache and run garbage collection before training loop
    torch.cuda.empty_cache()
    gc.collect()

    num_parameters = sum(p.numel() for p in model.parameters())
    print(f"Number of parameters: {num_parameters:.2e}.")

    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=15, gamma=0.75)

    # Data loaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    best_loss = float('inf')
    best_model = None

    train_losses = []
    val_losses = []
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    csi_scores = []

    # Training loop
    for epoch in range(1, num_epochs + 1):
        # Training step
        train_loss = training_unet(model, train_loader, optimizer, device=device, loss_f='BCE', water_threshold=0.5)
        train_losses.append(train_loss)

        # Validation step
        with torch.no_grad():  # Prevent gradient computation in validation
            val_loss, val_accuracy, val_precision, val_recall, val_f1_score, val_csi_score = validation_unet(
                model, val_loader, device=device, loss_f='BCE', water_threshold=0.5
            )

        # Move metrics to CPU and append to lists
        val_losses.append(val_loss)
        accuracies.append(val_accuracy)
        precisions.append(val_precision)
        recalls.append(val_recall)
        f1_scores.append(val_f1_score)
        csi_scores.append(val_csi_score)

        # Update learning rate
        scheduler.step()

        # Save best model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = copy.deepcopy(model).cpu()  # Move model to CPU for storage

        # Print progress
        if epoch % 1 == 0:
            print(f"Epoch: {epoch} | Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                  f"Best Validation Loss: {best_loss:.4f}")
            print(f"Metrics | Accuracy: {val_accuracy:.3f}, Precision: {val_precision:.3f}, Recall: {val_recall:.3f}, "
                  f"F1-score: {val_f1_score:.3f}, CSI-score: {val_csi_score:.3f}")
            print(f"Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

    # Cleanup
    del optimizer, val_loss, model, train_loader, train_loss
    gc.collect()
    torch.cuda.empty_cache()

    return best_loss, best_model, train_losses, val_losses, accuracies, precisions, recalls, f1_scores, csi_scores

In [ ]:
# Best Hyperparemeters
batch_size = 16
learning_rate = 0.01
init_hid_dim = 8
num_epochs = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_f = 'BCE'
machine = 'machine_1'

# Clear GPU memory before starting
torch.cuda.empty_cache()
gc.collect()

# Train and validate
print(f"Running training for Batch Size={batch_size}, Learning Rate={learning_rate}, Init Hid Dim={init_hid_dim}")

val_loss, best_model, train_losses, val_losses, accuracies, precisions, recalls, f1_scores, csi_scores = train_and_validate(
    batch_size=batch_size,
    learning_rate=learning_rate,
    init_hid_dim=init_hid_dim,
    num_epochs=num_epochs,
    device=device
)

# Clear GPU memory after training
torch.cuda.empty_cache()
gc.collect()

# Save the best model
save_model_path(
    machine=machine,
    model=best_model,  # No need to move to CPU if metrics are not tensors
    batch_size=batch_size,
    learning_rate=learning_rate,
    init_hid_dim=init_hid_dim,
    epochs=num_epochs,
    dir_output="model/models_trained"
)

# Save training and validation metrics
save_losses_metrics(
    machine=machine,
    train_losses=train_losses,
    val_losses=val_losses,
    metrics=[accuracies, precisions, recalls, f1_scores, csi_scores],
    batch_size=batch_size, 
    learning_rate=learning_rate, 
    init_hid_dim=init_hid_dim, 
    epochs=num_epochs,
    dir_output="model/losses_metrics" 
)

# Define the test loader
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Test the best model
with torch.no_grad():  # Disable gradient computation during testing
    model_loss = copy.deepcopy(best_model)
    test_loss, test_accuracy, test_precision, test_recall, test_f1_score, test_csi_score = validation_unet(
        model_loss, test_loader, device=device, loss_f=loss_f
    )

print(f'Average metrics for test dataset using model with best validation loss:\n\n\
BCE loss:          {test_loss:.3e}\n\
Accuracy:          {test_accuracy:.3f}\n\
Precision:         {test_precision:.3f}\n\
Recall:            {test_recall:.3f}\n\
F1 score:          {test_f1_score:.3f}\n\
CSI score:         {test_csi_score:.3f}')

# Cleanup to free memory
del best_model, model_loss, test_loader
gc.collect()
torch.cuda.empty_cache()